In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import time
import ipywidgets as wg
from ipywidgets import interact
from IPython.display import display
import networkx as nx
from src.environment import Environment
from src.runner import runner
from src.helpers import confidence_interval
from sklearn import preprocessing
import random
import json

# Calibration

## 1 Setting the parameters

In [2]:
age_groups = ['age_0_10', 'age_10_20', 'age_20_30', 'age_30_40', 'age_40_50', 
              'age_50_60', 'age_60_70', 'age_70_80', 'age_80_plus']

In [3]:
TIME = 120

In [4]:
parameters = {
    # general simulation parameters
    "time": TIME,
    "number_of_agents": 5000,
    "monte_carlo_runs": 1,
    
    # Cape Town specific parameters
    "total_initial_infections": [x for x in range(0, 19)], # total agents infected in CT
    "health_system_capacity": 0.0021, # relative (in terms of population) capacity of the hospitals
    
    # COVID-19 parameters
    "exposed_days": 4, # average number of days without symptoms and being able to infect others
    "asymptom_days": 10, # average number of days agents are infected but do not have symptoms
    "symptom_days": 10,# average number of days agents have mild symptoms
    "critical_days": 8, # average number of days agents are in critical condition
    "probability_symptomatic": 0.6165, # determines whether an agent will become asymptomatic or asymptomatic spreader
    "no_hospital_multiplier": 1.79, # the increase in probability if a critical agent cannot go to the hospital SOURCE: Zhou et al. 2020
    "probability_transmission": 0.05, # should be estimated to replicate realistic R0 number.
    
    "probability_critical": {key:value for key, value in zip(age_groups, [0.001, 0.003, 0.012, 0.032, 0.049, 0.102, 0.166, 0.244, 0.273])}, # probability that an agent enters a critical stage of the disease SOURCE: Verity et al.
    "probability_to_die": {key:value for key, value in zip(age_groups, [0.005, 0.021, 0.053, 0.126, 0.221, 0.303, 0.565, 0.653, 0.765])}, # probability to die per age group in critical stage SOURCE: Verity et al.
    
    # Policy parameters
    "lockdown_days" : [None for x in range(0, TIME)], # in the baseline this is 0, 5 march was the first reported case, 27 march was the start of the lockdown 35 days 
    "informality_dummy": 1.0, # setting this parameter at 0 will mean the lockdown is equally effective anywhere, alternative = 1
    
    # Specific policy parameters
    # (1) physical distancing measures such as increased hygiëne & face mask adoption 
    "physical_distancing_multiplier": 0.27, # Jarvis et al. 2020,
    # (2) reducing travel e.g. by reducing it for work, school or all
    "visiting_recurring_contacts_multiplier": 0.6, # depending on how strict the lockdown is at keeping you put.
    # (3) Testing and general awareness
    'likelihood_awareness': 0.6, # this will be increased through track & trace and coviid
    'self_isolation_multiplier': 0.4, # determines the percentage of connections cut thanks to self-isoluation can go up with coviid
    'aware_status': ['i2'], # i1 can be added if there is large scale testing 
    # (4) limiting mass contact e.g. forbidding large events
    "gathering_max_contacts": 6,
    
    # Technical parameters
    'init_infected_agent': 0, # to calculate R0
    "data_output": 'csv', # 'csv' or 'network', or 'False'
    
    # Depreciated paramters (can be used later)
    "probability_susceptible": 0.000, # probability that the agent will again be susceptible after having recovered
    
}

Next, we store these parameters in a .json file.

In [5]:
with open('parameters/parameters.json', 'w') as outfile:
    json.dump(parameters, outfile)

## 2 Generate input data files

Then, we generate a file that add per Ward (district) data on: 

- informality of the Ward,
- initial cases per ward.

to the already existing features in the population csv that has 

- distribution of population size across wards,
- the age distribution within wards.

In [6]:
informal_residential = pd.read_csv('input_data/Informal_Residential.csv').iloc[:-1]
inital_infections = pd.read_csv('input_data/Cases_With_Subdistricts.csv', index_col=0)
population = pd.read_csv('input_data/population.csv')

The informality score of the Wards is then normalized to make be able to compare them in the model and added to the population DataFrame.

In [7]:
x = informal_residential[['Informal_residential']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
informal_residential['Informal_residential'] = pd.DataFrame(x_scaled)

population['Informal_residential'] = informal_residential['Informal_residential']

Because the amount of agents are less than in the actual population of Cape Town, we filter some wards so that all of them are at least populated with an agent. Therefore, we calculate the smallest size of a neighbourhood should have as follows.

In [8]:
smallest_size = population['Population'].sum() / parameters['number_of_agents']

We then combine this data into one neighbourhood data set,

In [9]:
districts_data = []
for i in range(len(population)):
    if population['Population'].iloc[i] > smallest_size:
        districts_data.append(
            [int(population['WardID'].iloc[i]), {'Population': population['Population'].iloc[i],
                                                 'Density': population['Density'].iloc[i],
                                                 'lon': population['lon'].iloc[i],
                                                 'lat': population['lat'].iloc[i],
                                                 'Informal_residential': population['Informal_residential'].iloc[i],
                                                 'Cases_With_Subdistricts': inital_infections.loc[population['WardID'].iloc[i]]['Cases_03292020'],
                                                },
            ])
max_neighbourhoods = len(districts_data)

and export it to a json file. 

In [10]:
max_districts = len(districts_data) # this can be manually shortened to study dynamics in some districts
with open('parameters/district_data.json', 'w') as outfile:
    json.dump(districts_data[:max_districts], outfile)